In [5]:
import requests

In [6]:
import random

In [7]:
import time

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, ElementNotInteractableException, TimeoutException,JavascriptException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from bs4 import BeautifulSoup

# admin site project -----

In [34]:
url = "https://qa1-aws.oceaniacruises.com/admin/"

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# admin site project: identify if admin contains redirect (complete) - the redirects are as follow: /admin - /admin/ - ReturnUrl=

In [35]:
with requests.Session() as session:
    response = session.get(url, headers=headers, allow_redirects=False)
    print(response.status_code, response.headers['Location'])
    # redirect_url = response.headers["location"]

302 /loginredirector.aspx?ReturnUrl=%2fadmin%2f


# admin site project: handle the redirect by comparing my url with the current one (this can later be used to switch the POST to GET and back to POST)

In [ ]:
with requests.Session() as session:
    response = session.get(url, headers=headers, allow_redirects=True)
    if response.url =! url:
        print('not the same url')

In [15]:
response = requests.get(url, headers=headers)

In [16]:
soup = BeautifulSoup(response.text, "html.parser")

In [17]:
find_form = soup.find_all('form')

In [18]:
form_list = list(find_form)

In [19]:
for i, form in enumerate(form_list):
    #if 'country' not in form.prettify().lower():
    if 'input-575-218' in form.prettify().lower():
        print(f"Form {i}:\n{form.prettify()}\n\n")

Form 2:
<form autocomplete="off" class="c2_form form-material" data-js="signin-form">
 <div class="form-group">
  <!-- Error Message -->
  <div class="errorMessage hide" data-js="data-msg-error">
   <span>
    Sorry, we donât recognize that email and password combination. Please try again or recover your password.
   </span>
  </div>
 </div>
 <div class="form-group -icon">
  <label class="label-control" for="input-575-218">
   Email Address
  </label>
  <div class="input-group">
   <div class="input-group-prepend">
    <span class="input-group-text ogi-icon-envelope">
    </span>
   </div>
   <input class="form-control" data-msg="Please enter a valid email address." data-msg-customemail="Please enter a valid email address." data-msg-email="Please enter a valid email address." id="input-575-218" name="username" placeholder="Email Address" type="text"/>
   <!-- Error Message -->
   <div class="form-validation" data-container="#tooltip-f-745-803" data-js="form-validation" data-original-

-- cruise finder page, expand the drop down list check

In [19]:
chrome_options = Options()
chrome_options.add_argument("User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
chrome_options.add_argument("--headless=new")  # New headless implementation
chrome_options.add_argument("--disable-gpu")  # Disable GPU (useful for older environments)
chrome_options.add_argument("--window-size=1920,1080")  # Set window size for headless
chrome_options.add_argument("--no-sandbox")  # Bypass sandbox (useful in CI)
chrome_options.add_argument("--disable-dev-shm-usage")  # Prevent shared memory issues

In [15]:
driver = webdriver.Chrome(options=chrome_options)

-- testing out the urls function

In [34]:
driver = webdriver.Chrome(options=chrome_options)
url = "https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
def find_urls(driver, url):
    inner_html_list = []
    try:
        # Navigate to the page
        driver.get(url)

        # Wait for the page to load (adjust if necessary)
        time.sleep(10)

        # Use JavaScript to fetch the innerHTML of all elements with the given class name
        script = f"""
        const listItems = document.querySelectorAll('.c29_body_options_item');
        let hrefList = [];
        listItems.forEach(li => {{
            const anchor = li.querySelector('a'); // Search for <a> within each <li>
            if (anchor && anchor.href) {{
                hrefList.push(anchor.href);      // Push the href value to the list
            }}
        }});
        return hrefList;
        """
        
        # Execute the JavaScript and fetch the results
        inner_html_list = driver.execute_script(script)
        
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()
    
    return inner_html_list
print(find_urls(driver, url))

['https://preprod-aws.oceaniacruises.com/caribbean-cruises/miami-to-bridgetown-SIR241221/?sr=%2Fcruise-finder%23sort%3Dfeatured%3Adesc%26page%3D1%26pageSize%3D10', 'https://preprod-aws.oceaniacruises.com/cruise-finder#', 'https://preprod-aws.oceaniacruises.com/australia-cruises/sydney-to-sydney-REG250125/?sr=%2Fcruise-finder%23sort%3Dfeatured%3Adesc%26page%3D1%26pageSize%3D10', 'https://preprod-aws.oceaniacruises.com/cruise-finder#', 'https://preprod-aws.oceaniacruises.com/transatlantic-cruises/rio-de-janeiro-to-cape-town-INS250127/?sr=%2Fcruise-finder%23sort%3Dfeatured%3Adesc%26page%3D1%26pageSize%3D10', 'https://preprod-aws.oceaniacruises.com/cruise-finder#', 'https://preprod-aws.oceaniacruises.com/caribbean-cruises/bridgetown-to-miami-SIR250129/?sr=%2Fcruise-finder%23sort%3Dfeatured%3Adesc%26page%3D1%26pageSize%3D10', 'https://preprod-aws.oceaniacruises.com/cruise-finder#', 'https://preprod-aws.oceaniacruises.com/caribbean-cruises/miami-to-miami-SIR250301/?sr=%2Fcruise-finder%23sort

-- check if the cruises load and expand the dropdown menu for voyage listing

In [47]:
'''
try:
    url = "https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
    driver.get(url)
    time.sleep(12)
    # ensure the results loaded by checking the value of view button
    element = driver.find_element(By.CLASS_NAME, 'c265_aside_item')
    is_page_loaded = "0" not in element.get_attribute("outerText")
    # drop_down_class = ".btn.dropdown-toggle.btn-outline-black.btn-md" # Class of the dropdown - using CSS selector to acount for the spaces
    drop_down_xpath = "//button[contains(@class, 'btn') and contains(@class, 'dropdown-toggle') and contains(@class, 'btn-outline-black') and contains(@class, 'btn-md')]"  # XPath for the dropdown
    if is_page_loaded:
        print('Page loaded successfully')
        print(element.get_attribute("outerText"))  # Print the outerText of the element
        
        # Find the dropdown element directly without waiting
        drop_down_element = driver.find_element(By.XPATH, drop_down_xpath)
        
        # Scroll the dropdown element into view
        driver.execute_script("arguments[0].scrollIntoView(true);", drop_down_element)
        
        # Click the dropdown using JavaScript
        driver.execute_script("arguments[0].click();", drop_down_element)
        print('Dropdown clicked successfully')
        
        # Get the attribute aria-expanded
        drop_down_expanded = drop_down_element.get_attribute("aria-expanded")
        
        if drop_down_expanded.lower() == "true":
            print('Dropdown expanded successfully')
            print(drop_down_expanded)
        else:
            print('Dropdown did not expand successfully')
            print(drop_down_expanded)
    else:
        print('Page did not load successfully')

except WebDriverException as e:
    print('Error loading page:', e)

except ElementNotInteractableException as e:
    print('Element not interactable:', e)

finally:
    # Close the browser
    driver.quit()
'''

Page loaded successfully
VIEW 616 CRUISES
Dropdown clicked successfully
Dropdown expanded successfully
true
VIEW 616 CRUISES


-- extracting the urls from the page function , load urls to list

In [9]:
def find_urls(driver):
    urls_list = []
    try:
        urls = driver.find_elements(By.CLASS_NAME, 'c29_body_options')
        for url in urls:
            urls_list.append(url.get_attribute('href'))
    except WebDriverException as e:
        print('Error finding URLs:', e)
    return urls_list

-- below is to now select a value from the expanded list and selecting a value

In [33]:
'''
try:
    url = "https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
    driver.get(url)
    time.sleep(12)
    
    
    driver varibles 
 
    
    # view crusies parent class
    element = driver.find_element(By.CLASS_NAME, 'c265_aside_item')
    # drop down toggle class
    drop_down_xpath = "//button[@class='btn dropdown-toggle btn-outline-black btn-md']"
    drop_down_element = driver.find_element(By.XPATH, drop_down_xpath)
    # drop down individual values
    list_item = driver.find_element(By.XPATH, "//a[@id='bs-select-17-4']")
    # page navigation section
    page_navigation_bar = driver.find_element(By.XPATH, "//*[@class='listing -horizontal']")
    each_page = page_navigation_bar.find_elements(By.TAG_NAME, 'li')
    # initialize varibles to have global scope as its needed for the while loop to work
    page_amount = None
    current_page = None
    # ensure the results loaded by checking the value of view button
    is_page_loaded = "0" not in element.get_attribute("outerText")
    if is_page_loaded:
        print('Page loaded successfully')
        print(element.get_attribute("outerText"))  # Print the outerText of the element
        driver.execute_script("arguments[0].scrollIntoView(true);", drop_down_element)
        
        # Click the dropdown
        driver.execute_script("arguments[0].click();", drop_down_element)
        print('Dropdown clicked successfully')
        
        # Click the individual item selection
        driver.execute_script("arguments[0].click();", list_item) 
        print('Dropdown selection clicked successfully')
        
        # Wait for 20 seconds
        time.sleep(20)
        
        # lets check if the count updated
        if (drop_down_element.get_attribute("title") == "50"):
            print('Dropdown selection updated successfully')
            print(drop_down_element.get_attribute("title"))
            print(driver.current_url)
            url_list = find_urls(driver)
            print(url_list) # end
            # selecting the page navigation
            
            page_amount = len(each_page)
            current_page = page_navigation_bar.find_element(By.XPATH, "//*[@class='active']")
            

        else:
            print(drop_down_element.get_attribute("title"))
            print('Dropdown selection did not update successfully')
            print(driver.current_url)
    else:
        print('Page did not load successfully')

except WebDriverException as e:
    print('Error loading page:', e)

except ElementNotInteractableException as e:
    print('Element not interactable:', e)

finally:
    # Close the browser
    driver.quit()
'''

Page loaded successfully
VIEW 616 CRUISES
Dropdown clicked successfully
Dropdown expanded successfully
true
Dropdown selection clicked successfully
Dropdown selection updated successfully
50
https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=50


-- below will now incorporate a while loop to repeat the process for each page

In [16]:
try:
    url = "https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
    driver.get(url)
    time.sleep(12)
    # create global list to insert all the list into this one -- remeber to flatten at the end
    global_list_urls = []

    element = driver.find_element(By.CLASS_NAME, 'c265_aside_item')
    is_page_loaded = "0" not in element.get_attribute("outerText")
    if is_page_loaded:
        print('Page loaded successfully')
        # drop down toggle class
        drop_down_xpath = "//button[@class='btn dropdown-toggle btn-outline-black btn-md']"
        drop_down_element = driver.find_element(By.XPATH, drop_down_xpath)
        print(element.get_attribute("outerText"))  # Print the outerText of the element
        driver.execute_script("arguments[0].scrollIntoView(true);", drop_down_element)
        
        # Click the dropdown
        driver.execute_script("arguments[0].click();", drop_down_element)
        print('Dropdown clicked successfully')
        
        # Click the individual item selection
        # drop down individual values
        list_item = driver.find_element(By.XPATH, "//a[@id='bs-select-17-4']")
        driver.execute_script("arguments[0].click();", list_item) 
        print('Dropdown selection clicked successfully')
        
        # Wait for 20 seconds
        time.sleep(10)
        
        # lets check if the count updated
        if (drop_down_element.get_attribute("title") == "50"):
            print('Dropdown selection updated successfully')
            print(drop_down_element.get_attribute("title"))
            print(driver.current_url)
            url_list = find_urls(driver)
            print(url_list) # end

            '''
            # page navigation section
            page_navigation_bar = driver.find_element(By.XPATH, "//*[@class='listing -horizontal']")
            each_page = page_navigation_bar.find_elements(By.TAG_NAME, 'li')
            page_amount = len(each_page)
            current_page = page_navigation_bar.find_element(By.XPATH, "//*[@class='active']")
            print(page_amount)
            print(current_page)
            
            '''
        else:
            print(drop_down_element.get_attribute("title"))
            print('Dropdown selection did not update successfully')
            print(driver.current_url)
    else:
        print('Page did not load successfully')

except WebDriverException as e:
    print('Error loading page:', e)

except ElementNotInteractableException as e:
    print('Element not interactable:', e)

finally:
    # Close the browser
    driver.quit()

Page loaded successfully
VIEW 616 CRUISES
Dropdown clicked successfully
Dropdown selection clicked successfully
Dropdown selection updated successfully
50
https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=50
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [ ]:
'''
back up code
try:
    url = "https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
    driver.get(url)
    time.sleep(12)
    '''
    
    driver varibles 
 
    '''
    # view crusies parent class
    element = driver.find_element(By.CLASS_NAME, 'c265_aside_item')
    # drop down toggle class
    drop_down_xpath = "//button[@class='btn dropdown-toggle btn-outline-black btn-md']"
    drop_down_element = driver.find_element(By.XPATH, drop_down_xpath)
    # drop down individual values
    list_item = driver.find_element(By.XPATH, "//a[@id='bs-select-17-4']")
    # page navigation section
    page_navigation_bar = driver.find_element(By.XPATH, "//*[@class='listing -horizontal']")
    each_page = page_navigation_bar.find_elements(By.TAG_NAME, 'li')
    # initialize varibles to have global scope as its needed for the while loop to work
    page_amount = None
    current_page = None
    # create global list to insert all the list into this one -- remeber to flatten at the end
    global_list_urls = []

    while True:
        if int(current_page.text) > page_amount:
            print("completed all pages")
            break
        else:
            is_page_loaded = "0" not in element.get_attribute("outerText")
            if is_page_loaded:
                print('Page loaded successfully')
                print(element.get_attribute("outerText"))  # Print the outerText of the element
                driver.execute_script("arguments[0].scrollIntoView(true);", drop_down_element)
                
                # Click the dropdown
                driver.execute_script("arguments[0].click();", drop_down_element)
                print('Dropdown clicked successfully')
                
                # Click the individual item selection
                driver.execute_script("arguments[0].click();", list_item) 
                print('Dropdown selection clicked successfully')
                
                # Wait for 20 seconds
                time.sleep(20)
                
                # lets check if the count updated
                if (drop_down_element.get_attribute("title") == "50"):
                    print('Dropdown selection updated successfully')
                    print(drop_down_element.get_attribute("title"))
                    print(driver.current_url)
                    url_list = find_urls(driver)
                    print(url_list) # end
                    
                    # selecting the page navigation
                    page_amount = len(each_page)
                    current_page = page_navigation_bar.find_element(By.XPATH, "//*[@class='active']")
                    print(current_page)
                    
                    

                else:
                    print(drop_down_element.get_attribute("title"))
                    print('Dropdown selection did not update successfully')
                    print(driver.current_url)
            else:
                print('Page did not load successfully')

except WebDriverException as e:
    print('Error loading page:', e)

except ElementNotInteractableException as e:
    print('Element not interactable:', e)

finally:
    # Close the browser
    driver.quit()

'''